In [1]:
import requests as rq
import re
from bs4 import BeautifulSoup
import io
import time
from collections import Counter
import sys

In [2]:
date_list = [] #日期( without '/' )
title_href =[] #標題與url
bomb_list = [] #推文數
claim_list = [] #公告類文章
ids = []
push = []
ids_comments_count = []
boo_comments = []
push_comments = []
comments = []
stop = []
#a_img = [] #內文圖片url list
keyword_url_list = []
k_del_not_url = []

In [3]:
url ='https://www.ptt.cc/bbs/Beauty/index.html'
#2019第一頁（要忽略12/31的）
url_start = 'https://www.ptt.cc/bbs/Beauty/index2748.html'
#2019最後一頁(要忽略1/1的)
url_end = 'https://www.ptt.cc/bbs/Beauty/index3143.html'

In [4]:
#解決開頭同意已滿18問題
def over_18(url):
    payload={
        'from':url,
        'yes':'yes'
    }
    rs = rq.session()
    res = rs.post('https://www.ptt.cc/ask/over18?from=%2Fbbs%2FBeauty%2Findex.html',data = payload)
    res = rs.get(url)
    content = res.text
    return content

In [5]:
def top_n_scores(n, score_dict):
  lot = [(k,v) for k, v in score_dict.items()] #make list of tuple from scores dict
  nl = []
  while len(lot)> 0:
      nl.append(max(lot, key=lambda x: x[1]))
      lot.remove(nl[-1])
  return nl[0:n] 

In [6]:
def checkformat(soup, class_tag, data, index, url):

    try:

        content = soup.select(class_tag)[index].text

    except:

        content='null'

    return content


In [7]:
#抓出網址,時間,推文數
def get_all_href(url):
    soup = BeautifulSoup(url, 'html.parser')
    results = soup.select("div.title")
    dates = soup.select('div.date')
    p_b_counts = soup.select('div.nrec')#推文數
    #print(dates)
    #print('dates長度',len(dates),'\n')
    #print(results,'\n')
    #print('title數',len(results),'\n')
    claims = soup.find_all("a", string=re.compile("公告"))
    for claim in claims:
        if claim not in claim_list:
            claim_list.append(claim)

    for date in dates:
        if date != None:
            date_list.append(date.string.replace('/', ''))
    #print((date_list))
    for item in results:
        a_item = item.select_one("a")
        if a_item == None:
            break
        else:
            title_href.append(a_item.string)
            title_href.append('https://www.ptt.cc' + a_item.get('href'))
    for p_b in p_b_counts:
        bomb_list.append(p_b.string)
    #print(bomb_list)
    #print(len(bomb_list))

In [8]:
#進入文章
def get_push_boo_info(url):
    wordlist = []
    soup = BeautifulSoup(url, 'html.parser')
    words = soup.select('span',class_ = 'f2')
    pushes = soup.select('span.hl.push-tag')
    user_id = soup.select('span.f3.hl.push-userid')
    all_info = soup.select('div.push')
    #print(user_id)
    for word in words:
        wordlist.append(word)
    #忽略沒有綠文章網址
    for s in wordlist:
        if '※ 文章網址: ' in str(s):
            stop.append('no stop')
            break
    if 'no stop' not in stop:
        return
    #找發信站 
    temp=[]
    for s in wordlist:
        if '發信站' in str(s):
            temp.append(s)
            break
    #print(temp[0])
    if temp ==None:#忽略沒有發信站的文章
        return
    
    for p in pushes:
        push.append(p.string)
        #print(p.string)
        if p.string == '推 ':
            push_comments.append(p.string)
        elif p.string == '→ ':
            comments.append(p.string)
        elif p.string == '噓 ':
            boo_comments.append(p.string)
    
    for user in user_id:
        ids.append(user.string)

In [9]:
#1-1
#換頁，跑index範圍內所有文章資訊
def Crawl():
    for page in range(0,396):
        update_url = 'https://www.ptt.cc/bbs/Beauty/index' + str(2748 + page) +'.html'
        print(update_url)
        #delay time 如果不能存取再調大～
        time.sleep(0.01) 
        update_url_ok = over_18(update_url)
        ###
        get_all_href(url = update_url_ok)   
    #資料處理
    #組合標題跟網址
    title_href_t = [title_href[i:i+2] for i in range(0,len(title_href),2)]
    
    #刪除第一頁的12/31
    del title_href_t[0:17]
    del date_list[0:17]
    del bomb_list[0:17]

    #刪除最後一頁的1/1
    del title_href_t[len(title_href_t)-15:len(title_href_t)]
    del date_list[len(date_list)-15:len(date_list)]
    del bomb_list[len(bomb_list)-15:len(bomb_list)]

    #刪除公告文
    claim_list1 = [] #只有公告字串標題
    for claim in claim_list:
        claim = claim.string
        claim_list1.append(claim)
    for i in range(0,len(title_href_t)-len(claim_list)):
        for j in range(0,len(claim_list1)):
            if claim_list1[j] in title_href_t[i][0]:
                del title_href_t[i]
                del date_list[i]
                del bomb_list[i]
    #第一題的輸出檔案
    #寫出檔案all_articles.txt
    article_count = 0
    bomb_count = 0
    for i in range(0,len(date_list)):
        article_count += 1
        articles=open("all_articles.txt",'a')
        print(str(date_list[i])+','+str(title_href_t[i][0]) +',' + str(title_href_t[i][1]),file=articles)
        #articles.write('date_list[i]+','+title_href_t[i][0] +',' + title_href_t[i][1]\n')
        articles.close()
    #寫出爆文檔案 all_popular.txt
        if bomb_list[i] == '爆':
            bomb_count += 1
            popular=open("all_popular.txt",'a')
            print(str(date_list[i])+','+str(title_href_t[i][0]) +',' + str(title_href_t[i][1]),file=popular)
            #popular.write('date_list[i]+','+ title_href_t[i][0] +',' + title_href_t[i][1]\n')
            popular.close()

In [10]:
#1-2
#輸入開始結束日期
#開爬完的all_articles.txt去用
def Push(start_date,end_date):    
    total_list = []
    total_list_true = []
    date_list_true = []
    title_list_true = []
    url_list_true = []
    with open( './all_articles.txt', 'r' ) as f:
        for line in f.readlines():
            total_list.append(str(line.strip()))
        for i in range(0,len(total_list)):
            total_list_true.append(str(total_list[i]).split(','))
        for i in range(0,len(total_list_true)):
            date_list_true.append(total_list_true[i][0])
            title_list_true.append(total_list_true[i][1])
            url_list_true.append(total_list_true[i][2])
        #print(date_list_true.index(str(101))

    #開完檔案 分完日期標題網址
        if str(start_date) not in date_list_true or str(end_date) not in date_list_true:
            print('不符合，重新輸入一次！')
        elif str(start_date) in date_list_true and str(end_date) in date_list_true:
            start_date_index = date_list_true.index(str(start_date))
            end_date_index = date_list_true.index(str(end_date))
            if start_date_index < end_date_index:
                print('ok')
            elif start_date_index == end_date_index:
                while date_list_true[start_date_index] == date_list_true[end_date_index]:
                    end_date_index = end_date_index + 1
            else:
                print('不符合，重新輸入一次！')
        f.close()

    for i in range(start_date_index,end_date_index):
        update_url = url_list_true[i]
        print(update_url)
        #delay time 如果不能存取再調大～
        time.sleep(0.01) 
        update_url_ok = over_18(update_url)
        get_push_boo_info(url = update_url_ok)
    #print(len(ids),len(push))
    ids_comments_count=list(zip(ids,push))  
    #print('總推數：',len(push_comments),'\n'+'總箭頭數：',len(comments),'\n'+'總噓數：',len(boo_comments))
    
    
    #排名推噓文前十的id及數量
    s_e_push = []
    s_e_boo = []
    for i in range(0,len(ids_comments_count)):
        if ids_comments_count[i][1] == '推 ':
            s_e_push.append(ids_comments_count[i])
        elif ids_comments_count[i][1] == '噓 ':
            s_e_boo.append(ids_comments_count[i])

    push1 = Counter(s_e_push)
    boo = Counter(s_e_boo)
    #print(push)


    top30_push = list(top_n_scores(len(push1), push1))  
    top30_boo = list(top_n_scores(len(boo),boo))



    #整理出id字串list 
    top30_boo_id = []
    top30_push_id = []
    top30_b_id = [] #純id list
    top30_p_id = [] #純id list
    top30_b_c = []  #純數字list
    top30_p_c = []  #純數字list
    for i in range(0,len(top30_boo)):
        top30_boo_id.append(top30_boo[i][0])
    for i in range(0,len(top30_boo_id)):
        top30_b_id.append(top30_boo_id[i][0])
    #print(top30_b_id)

    for i in range(0,len(top30_push)):
        top30_push_id.append(top30_push[i][0])
    for i in range(0,len(top30_push_id)):
        top30_p_id.append(top30_push_id[i][0])
    #print(top30_p_id)

    #########

    for i in range(0,len(top30_boo)):
        top30_b_c.append(top30_boo[i][1])
    #print(top30_b_c)

    for i in range(0,len(top30_push)):
        top30_p_c.append(top30_push[i][1])
    #print(top30_p_c)


    #=====================sort id====================== 
    top30_p = list(zip(top30_p_id,top30_p_c))
    #print(top30_p)
    top30_p = sorted(top30_p,key = lambda x:(-x[1],x[0]))
    #print(top30_p)



    top30_b = list(zip(top30_b_id,top30_b_c))
    #print(top30_b)
    top30_b = sorted(top30_b,key = lambda x:(-x[1],x[0]))
    #print(top30_b)


    top10_p = top30_p[0:10]
    #print(top10_p)
    top10_b = top30_b[0:10]
    #print(top10_b)
    
    #輸出push檔
    pushfile=open('push[' + str(start_date) + '-' + str(end_date) + ']' + '.txt','a')
    print('all like:',len(push_comments),file = pushfile)
    print('all boo:',len(boo_comments),file = pushfile)
    for i in range(0,len(top10_p)):
        pushfile=open('push[' + str(start_date) + '-' + str(end_date) + ']' + '.txt','a')
        print('like '+'#'+str((i+1))+':'+' '+str(top10_p[i][0])+' '+str(top10_p[i][1]),file=pushfile)
        #pushfile.write()'like '+'#'+str((i+1))+':'+' '+str(top10_p[i][0])+' '+str(top10_p[i][1]),file=pushfile)
        pushfile.close()
    for i in range(0,len(top10_b)):
        pushfile=open('push[' + str(start_date) + '-' + str(end_date) + ']' + '.txt','a')
        print('boo '+'#'+str((i+1))+':'+' '+str(top10_b[i][0])+' '+str(top10_b[i][1]),file=pushfile)
        pushfile.close()

In [11]:
#1-3 popular
#輸入開始結束日期
#開爬完的all_articles.txt去用
def Popular(start_date,end_date):
    total_list = []
    total_list_true = []
    date_list_true = []
    title_list_true = []
    url_list_true = []
    #a_img = []
    with open( './all_articles.txt', 'r' ) as f:
        for line in f.readlines():
            total_list.append(str(line.strip()))
        for i in range(0,len(total_list)):
            total_list_true.append(str(total_list[i]).split(','))
        for i in range(0,len(total_list_true)):
            date_list_true.append(total_list_true[i][0])
            title_list_true.append(total_list_true[i][1])
            url_list_true.append(total_list_true[i][2])
        #print(date_list_true.index(str(101))

    #開完檔案 分完日期標題網址
        if str(start_date) not in date_list_true or str(end_date) not in date_list_true:
            print('不符合，重新輸入一次！')
        elif str(start_date) in date_list_true and str(end_date) in date_list_true:
            start_date_index = date_list_true.index(str(start_date))
            end_date_index = date_list_true.index(str(end_date))
            if start_date_index < end_date_index:
                print('ok')
            elif start_date_index == end_date_index:
                while date_list_true[start_date_index] == date_list_true[end_date_index]:
                    end_date_index = end_date_index + 1

            else:
                print('不符合，重新輸入一次！')
        f.close()
        
    p_total_list = []
    p_total_list_true = []
    p_date_list_true = []
    p_title_list_true = []
    p_url_list_true = []
    s_e_boom = []  
    s_e_boom_split = [] #範圍內爆文列表(分割後)
    with open( './all_popular.txt', 'r' ) as f:
        for line in f.readlines():
            p_total_list.append(str(line.strip()))
        for i in range(0,len(p_total_list)):
            p_total_list_true.append(str(p_total_list[i]).split(','))
        for i in range(0,len(p_total_list_true)):
            p_date_list_true.append(p_total_list_true[i][0])
            p_title_list_true.append(p_total_list_true[i][1])
            p_url_list_true.append(p_total_list_true[i][2])
        #輸出爆文
        for i in range(start_date_index,end_date_index):
            if title_list_true[i] in p_title_list_true:
                s_e_boom.append(total_list[i])
        for i in range(0,len(s_e_boom)):
            s_e_boom_split.append(str(s_e_boom[i]).split(','))
        f.close()
    #print(s_e_boom_split)
    
    #找出爆文內的圖片url
    a_img = []
    for i in range(0,len(s_e_boom_split)):
        update_url = s_e_boom_split[i][2]
        print(update_url)
        #delay time 如果不能存取再調大～
        time.sleep(0.01) 
        update_url_ok = over_18(update_url)
        wordlist = []
        soup = BeautifulSoup(update_url_ok,'html.parser')
        pics = soup.select("a",target = "_blank")
        words = soup.select('span',class_ = 'f2')
        for word in words:
            wordlist.append(word)
        #忽略沒有綠文章網址
        for s in wordlist:
            #print(s)
            if '※ 文章網址: ' in str(s):
                stop.append('no stop')
                break
        #print(stop)
        if 'no stop' not in stop:
            continue
        #找發信站 
        temp=[]
        for s in wordlist:
            if '發信站' in str(s):
                temp.append(s)
                break
        #print(temp)
        if temp ==None:#忽略沒有發信站的文章
            continue

        a=[]

        for s in pics:
            a.append(s["href"])
        #print(a)
        for i in range(0,len(a)):
            if 'https://i.imgur' in a[i]:
                a_img.append(a[i])
    #print(a_img)
    #輸出popular檔
    popular_url=open('popular[' + str(start_date) + '-' + str(end_date) + ']' + '.txt','a')
    print('number of popular articles: ',len(s_e_boom_split),file=popular_url)
    #popular_url.close()
    for i in range(0,len(a_img)): 
        popular_url=open('popular[' + str(start_date) + '-' + str(end_date) + ']' + '.txt','a')
        print(a_img[i] ,file=popular_url)
        popular_url.close()

In [12]:
#1-4 keyword
#輸入開始結束日期與keyword 
def Keyword(keyword,start_date,end_date):
    total_list = []
    total_list_true = []
    date_list_true = []
    title_list_true = []
    url_list_true = []
    with open( './all_articles.txt', 'r' ) as f:
        for line in f.readlines():
            total_list.append(str(line.strip()))
        for i in range(0,len(total_list)):
            total_list_true.append(str(total_list[i]).split(','))
        for i in range(0,len(total_list_true)):
            date_list_true.append(total_list_true[i][0])
            title_list_true.append(total_list_true[i][1])
            url_list_true.append(total_list_true[i][2])
        #print(date_list_true.index(str(101))
    #開完檔案 分完日期標題網址
    
        if str(start_date) not in date_list_true or str(end_date) not in date_list_true:
            print('不符合，重新輸入一次！')
        elif str(start_date) in date_list_true and str(end_date) in date_list_true:
            start_date_index = date_list_true.index(str(start_date))
            end_date_index = date_list_true.index(str(end_date))
            if start_date_index < end_date_index:
                print('ok')
            elif start_date_index == end_date_index:
                while date_list_true[start_date_index] == date_list_true[end_date_index]:
                    end_date_index = end_date_index + 1
            else:
                print('不符合，重新輸入一次！')
        f.close()
    #找出範圍內 且含有keyword的文章
    #找出文章內的圖片url
    for i in range(start_date_index,end_date_index):
        update_url = total_list_true[i][2]
        print(update_url)
        #delay time 如果不能存取再調大～
        time.sleep(0.01) 
        update_url_ok = over_18(update_url)
        
        
        wordlist = []
        wordlist1=[]
        temp_pics=[]
        stop = []
        soup = BeautifulSoup(update_url_ok, 'html.parser')
        words = soup.select('span',class_ = 'f2')
        pics = soup.select("a",target = "_blank")

        for word in words:
            wordlist.append(word)
            #忽略沒有綠文章網址
 
        for word in words:
            wordlist1.append(word.string)
        for s in wordlist:
            if '※ 文章網址: ' in str(s):
                stop.append('no stop')
                break
            
        #####
        if 'no stop' not in stop:
            continue
        for word in words:
            wordlist1.append(word.string)
        #找發信站 
        temp=[]
        for s in wordlist1:
            if '發信站' in str(s):
                temp.append(s)
                break
        #print(temp[0])
        if temp ==None:#忽略沒有發信站的文章
            continue
        
        i = wordlist1.index(str(temp[0]))
        wordlist1 = wordlist1[0:i]

        for s in pics:
            temp_pics.append(s["href"])

        for i in range(0,len(temp_pics)):
            if 'https://i.imgur'  in temp_pics[i]:
                wordlist1.append(temp_pics[i])

        #####
        
        date = checkformat(soup, '.article-meta-value', 'date', 3, update_url_ok)
        content = soup.find(id="main-content").text
        target_content = u'※ 發信站: 批踢踢實業坊(ptt.cc),'
        #去除掉 target_content
        content = content.split(target_content)
        content = content[0].split(date)

        #去除掉文末 --
        main_content = content[1].replace('--', '')
        #印出內文
        #print(main_content)
    
    
    
    #check keywords
        if keyword in main_content or keyword in wordlist1: 
            for u in range(0,len(temp_pics)):
                keyword_url_list.append(temp_pics[u])
        
        
        
    for i in range(0,len(keyword_url_list)):

        if keyword_url_list[i][0] != '/' :
            if 'https://i.imgur' in str(keyword_url_list[i]):
                k_del_not_url.append(keyword_url_list[i])        
    #print(k_del_not_url) 
    
    #輸出keyword檔
    for i in range(0,len(k_del_not_url)): 
        keyword_url=open('keyword(' + str(keyword) + ')' + '[' + str(start_date) + '-' +str(end_date)+ ']' + '.txt','a')
        print(k_del_not_url[i],file=keyword_url)
        keyword_url.close()

In [13]:
#輸入來選擇功能

if sys.argv[1] == 'crawl':
    print('Crawl')
    Crawl()
elif sys.argv[1] == 'push':
    print('Push')
    Push(int(sys.argv[2]), int(sys.argv[3]))
elif sys.argv[1] == 'popular':
    print('Popular')
    Popular(int(sys.argv[2]), int(sys.argv[3]))
elif sys.argv[1] == 'keyword':
    print('Keyword')
    Keyword(sys.argv[2], int(sys.argv[3]), int(sys.argv[4]))



In [16]:
Push(start_date = 201,end_date = 301)

ok
https://www.ptt.cc/bbs/Beauty/M.1548951998.A.B74.html
https://www.ptt.cc/bbs/Beauty/M.1548953159.A.886.html
https://www.ptt.cc/bbs/Beauty/M.1548953353.A.2FC.html
https://www.ptt.cc/bbs/Beauty/M.1548955472.A.701.html
https://www.ptt.cc/bbs/Beauty/M.1548957996.A.0DF.html
https://www.ptt.cc/bbs/Beauty/M.1548959184.A.67A.html
https://www.ptt.cc/bbs/Beauty/M.1548960557.A.532.html
https://www.ptt.cc/bbs/Beauty/M.1548960717.A.41F.html
https://www.ptt.cc/bbs/Beauty/M.1548965951.A.427.html
https://www.ptt.cc/bbs/Beauty/M.1548976825.A.360.html
https://www.ptt.cc/bbs/Beauty/M.1548984884.A.877.html
https://www.ptt.cc/bbs/Beauty/M.1548985210.A.E6A.html
https://www.ptt.cc/bbs/Beauty/M.1548997135.A.18E.html
https://www.ptt.cc/bbs/Beauty/M.1548997203.A.F9F.html
https://www.ptt.cc/bbs/Beauty/M.1549015971.A.D51.html
https://www.ptt.cc/bbs/Beauty/M.1549019569.A.3B0.html
https://www.ptt.cc/bbs/Beauty/M.1549024385.A.C5E.html
https://www.ptt.cc/bbs/Beauty/M.1549027236.A.1D8.html
https://www.ptt.cc/bbs/Be

https://www.ptt.cc/bbs/Beauty/M.1549592107.A.FA1.html
https://www.ptt.cc/bbs/Beauty/M.1549595156.A.491.html
https://www.ptt.cc/bbs/Beauty/M.1549596876.A.C2E.html
https://www.ptt.cc/bbs/Beauty/M.1549598176.A.FDE.html
https://www.ptt.cc/bbs/Beauty/M.1549598470.A.2E8.html
https://www.ptt.cc/bbs/Beauty/M.1549608541.A.8BA.html
https://www.ptt.cc/bbs/Beauty/M.1549611003.A.548.html
https://www.ptt.cc/bbs/Beauty/M.1549611124.A.C4E.html
https://www.ptt.cc/bbs/Beauty/M.1549618862.A.E74.html
https://www.ptt.cc/bbs/Beauty/M.1549624955.A.E71.html
https://www.ptt.cc/bbs/Beauty/M.1549625552.A.1E3.html
https://www.ptt.cc/bbs/Beauty/M.1549629691.A.D7A.html
https://www.ptt.cc/bbs/Beauty/M.1549630471.A.766.html
https://www.ptt.cc/bbs/Beauty/M.1549630957.A.376.html
https://www.ptt.cc/bbs/Beauty/M.1549632797.A.C82.html
https://www.ptt.cc/bbs/Beauty/M.1549635399.A.96C.html
https://www.ptt.cc/bbs/Beauty/M.1549636849.A.68F.html
https://www.ptt.cc/bbs/Beauty/M.1549640864.A.3D2.html
https://www.ptt.cc/bbs/Beaut

https://www.ptt.cc/bbs/Beauty/M.1550126468.A.DCF.html
https://www.ptt.cc/bbs/Beauty/M.1550131169.A.D56.html
https://www.ptt.cc/bbs/Beauty/M.1550134159.A.99F.html
https://www.ptt.cc/bbs/Beauty/M.1550143413.A.7C6.html
https://www.ptt.cc/bbs/Beauty/M.1550147686.A.C6A.html
https://www.ptt.cc/bbs/Beauty/M.1550148397.A.67E.html
https://www.ptt.cc/bbs/Beauty/M.1550150436.A.5DB.html
https://www.ptt.cc/bbs/Beauty/M.1550153023.A.3C9.html
https://www.ptt.cc/bbs/Beauty/M.1550153133.A.884.html
https://www.ptt.cc/bbs/Beauty/M.1550160706.A.442.html
https://www.ptt.cc/bbs/Beauty/M.1550161171.A.95A.html
https://www.ptt.cc/bbs/Beauty/M.1550162558.A.B5E.html
https://www.ptt.cc/bbs/Beauty/M.1550163164.A.8D0.html
https://www.ptt.cc/bbs/Beauty/M.1550195600.A.773.html
https://www.ptt.cc/bbs/Beauty/M.1550200484.A.1DE.html
https://www.ptt.cc/bbs/Beauty/M.1550202242.A.13D.html
https://www.ptt.cc/bbs/Beauty/M.1550202978.A.5B4.html
https://www.ptt.cc/bbs/Beauty/M.1550204803.A.885.html
https://www.ptt.cc/bbs/Beaut

https://www.ptt.cc/bbs/Beauty/M.1550748216.A.8CC.html
https://www.ptt.cc/bbs/Beauty/M.1550750396.A.F59.html
https://www.ptt.cc/bbs/Beauty/M.1550755285.A.5B8.html
https://www.ptt.cc/bbs/Beauty/M.1550756290.A.B12.html
https://www.ptt.cc/bbs/Beauty/M.1550758981.A.979.html
https://www.ptt.cc/bbs/Beauty/M.1550759410.A.058.html
https://www.ptt.cc/bbs/Beauty/M.1550762648.A.F16.html
https://www.ptt.cc/bbs/Beauty/M.1550768078.A.E06.html
https://www.ptt.cc/bbs/Beauty/M.1550769563.A.3F6.html
https://www.ptt.cc/bbs/Beauty/M.1550772613.A.4E8.html
https://www.ptt.cc/bbs/Beauty/M.1550798801.A.855.html
https://www.ptt.cc/bbs/Beauty/M.1550801596.A.F79.html
https://www.ptt.cc/bbs/Beauty/M.1550805523.A.4D9.html
https://www.ptt.cc/bbs/Beauty/M.1550808888.A.F0C.html
https://www.ptt.cc/bbs/Beauty/M.1550815029.A.CE9.html
https://www.ptt.cc/bbs/Beauty/M.1550820961.A.0F2.html
https://www.ptt.cc/bbs/Beauty/M.1550824425.A.ED7.html
https://www.ptt.cc/bbs/Beauty/M.1550825783.A.3BC.html
https://www.ptt.cc/bbs/Beaut

In [17]:
Push(301,315)

ok
https://www.ptt.cc/bbs/Beauty/M.1551369712.A.A79.html
https://www.ptt.cc/bbs/Beauty/M.1551370308.A.FAB.html
https://www.ptt.cc/bbs/Beauty/M.1551370346.A.B64.html
https://www.ptt.cc/bbs/Beauty/M.1551371071.A.E13.html
https://www.ptt.cc/bbs/Beauty/M.1551373463.A.844.html
https://www.ptt.cc/bbs/Beauty/M.1551375989.A.BAC.html
https://www.ptt.cc/bbs/Beauty/M.1551395484.A.DB2.html
https://www.ptt.cc/bbs/Beauty/M.1551395738.A.964.html
https://www.ptt.cc/bbs/Beauty/M.1551399200.A.8A8.html
https://www.ptt.cc/bbs/Beauty/M.1551408232.A.AD3.html
https://www.ptt.cc/bbs/Beauty/M.1551413244.A.32C.html
https://www.ptt.cc/bbs/Beauty/M.1551418965.A.ECC.html
https://www.ptt.cc/bbs/Beauty/M.1551424824.A.6E0.html
https://www.ptt.cc/bbs/Beauty/M.1551426914.A.1B7.html
https://www.ptt.cc/bbs/Beauty/M.1551429447.A.5B7.html
https://www.ptt.cc/bbs/Beauty/M.1551429913.A.129.html
https://www.ptt.cc/bbs/Beauty/M.1551445787.A.D11.html
https://www.ptt.cc/bbs/Beauty/M.1551446619.A.DEF.html
https://www.ptt.cc/bbs/Be

https://www.ptt.cc/bbs/Beauty/M.1551960854.A.4D9.html
https://www.ptt.cc/bbs/Beauty/M.1551961664.A.FC8.html
https://www.ptt.cc/bbs/Beauty/M.1551963002.A.C32.html
https://www.ptt.cc/bbs/Beauty/M.1551970087.A.1E2.html
https://www.ptt.cc/bbs/Beauty/M.1551970633.A.FDA.html
https://www.ptt.cc/bbs/Beauty/M.1551979306.A.037.html
https://www.ptt.cc/bbs/Beauty/M.1551980382.A.F33.html
https://www.ptt.cc/bbs/Beauty/M.1552012043.A.D52.html
https://www.ptt.cc/bbs/Beauty/M.1552019288.A.AE4.html
https://www.ptt.cc/bbs/Beauty/M.1552019695.A.C8F.html
https://www.ptt.cc/bbs/Beauty/M.1552022671.A.6CF.html
https://www.ptt.cc/bbs/Beauty/M.1552025375.A.C24.html
https://www.ptt.cc/bbs/Beauty/M.1552026093.A.5D6.html
https://www.ptt.cc/bbs/Beauty/M.1552029723.A.3D0.html
https://www.ptt.cc/bbs/Beauty/M.1552033679.A.30E.html
https://www.ptt.cc/bbs/Beauty/M.1552033877.A.74C.html
https://www.ptt.cc/bbs/Beauty/M.1552034145.A.F3E.html
https://www.ptt.cc/bbs/Beauty/M.1552035603.A.4F1.html
https://www.ptt.cc/bbs/Beaut

MissingSchema: Invalid URL '都跟廣告一樣性感嗎': No schema supplied. Perhaps you meant http://都跟廣告一樣性感嗎?

In [18]:
Push(701,701)

https://www.ptt.cc/bbs/Beauty/M.1561916397.A.715.html
https://www.ptt.cc/bbs/Beauty/M.1561925911.A.F99.html
https://www.ptt.cc/bbs/Beauty/M.1561943766.A.64E.html
https://www.ptt.cc/bbs/Beauty/M.1561946214.A.73A.html
https://www.ptt.cc/bbs/Beauty/M.1561948664.A.8E1.html
https://www.ptt.cc/bbs/Beauty/M.1561951323.A.AB1.html
https://www.ptt.cc/bbs/Beauty/M.1561960916.A.283.html
https://www.ptt.cc/bbs/Beauty/M.1561961340.A.93D.html
https://www.ptt.cc/bbs/Beauty/M.1561961348.A.2A1.html
https://www.ptt.cc/bbs/Beauty/M.1561962880.A.CE3.html
https://www.ptt.cc/bbs/Beauty/M.1561964511.A.CB5.html
https://www.ptt.cc/bbs/Beauty/M.1561965155.A.6F2.html
https://www.ptt.cc/bbs/Beauty/M.1561965367.A.657.html
https://www.ptt.cc/bbs/Beauty/M.1561968836.A.641.html
https://www.ptt.cc/bbs/Beauty/M.1561970645.A.2B1.html
https://www.ptt.cc/bbs/Beauty/M.1561982682.A.E50.html
https://www.ptt.cc/bbs/Beauty/M.1561983124.A.350.html
https://www.ptt.cc/bbs/Beauty/M.1561983488.A.216.html
https://www.ptt.cc/bbs/Beaut

In [19]:
Popular(201,301)

ok
https://www.ptt.cc/bbs/Beauty/M.1549112045.A.7E4.html
https://www.ptt.cc/bbs/Beauty/M.1549191634.A.2B2.html
https://www.ptt.cc/bbs/Beauty/M.1549244450.A.C74.html
https://www.ptt.cc/bbs/Beauty/M.1549339754.A.B47.html
https://www.ptt.cc/bbs/Beauty/M.1549453159.A.36E.html
https://www.ptt.cc/bbs/Beauty/M.1549509439.A.8DE.html
https://www.ptt.cc/bbs/Beauty/M.1549582662.A.755.html
https://www.ptt.cc/bbs/Beauty/M.1549722005.A.A9D.html
https://www.ptt.cc/bbs/Beauty/M.1549875682.A.38E.html
https://www.ptt.cc/bbs/Beauty/M.1549896865.A.46C.html
https://www.ptt.cc/bbs/Beauty/M.1550117991.A.68F.html
https://www.ptt.cc/bbs/Beauty/M.1550148397.A.67E.html
https://www.ptt.cc/bbs/Beauty/M.1550153133.A.884.html
https://www.ptt.cc/bbs/Beauty/M.1550399989.A.6D7.html
https://www.ptt.cc/bbs/Beauty/M.1550458614.A.DB2.html
https://www.ptt.cc/bbs/Beauty/M.1550554524.A.470.html
https://www.ptt.cc/bbs/Beauty/M.1550654239.A.493.html
https://www.ptt.cc/bbs/Beauty/M.1550841454.A.2C7.html
https://www.ptt.cc/bbs/Be

In [20]:
Popular(301,315)

ok
https://www.ptt.cc/bbs/Beauty/M.1551452416.A.957.html
https://www.ptt.cc/bbs/Beauty/M.1551595597.A.C45.html
https://www.ptt.cc/bbs/Beauty/M.1551794536.A.906.html
https://www.ptt.cc/bbs/Beauty/M.1551851024.A.A11.html
https://www.ptt.cc/bbs/Beauty/M.1551925424.A.A8B.html
https://www.ptt.cc/bbs/Beauty/M.1551947402.A.4DD.html
https://www.ptt.cc/bbs/Beauty/M.1552226649.A.680.html
https://www.ptt.cc/bbs/Beauty/M.1552275410.A.0AA.html
https://www.ptt.cc/bbs/Beauty/M.1552432476.A.1BE.html
https://www.ptt.cc/bbs/Beauty/M.1552559044.A.7D4.html


In [21]:
Popular(701,701)

https://www.ptt.cc/bbs/Beauty/M.1561968836.A.641.html


In [23]:
Keyword('新年快樂',101,115)

ok
https://www.ptt.cc/bbs/Beauty/M.1546329828.A.B91.html
https://www.ptt.cc/bbs/Beauty/M.1546330983.A.A9A.html
https://www.ptt.cc/bbs/Beauty/M.1546331168.A.296.html
https://www.ptt.cc/bbs/Beauty/M.1546331206.A.33E.html
https://www.ptt.cc/bbs/Beauty/M.1546331399.A.D17.html
https://www.ptt.cc/bbs/Beauty/M.1546337668.A.17E.html
https://www.ptt.cc/bbs/Beauty/M.1546344370.A.9D5.html
https://www.ptt.cc/bbs/Beauty/M.1546347753.A.656.html
https://www.ptt.cc/bbs/Beauty/M.1546355051.A.BD4.html
https://www.ptt.cc/bbs/Beauty/M.1546355630.A.26C.html
https://www.ptt.cc/bbs/Beauty/M.1546357465.A.142.html
https://www.ptt.cc/bbs/Beauty/M.1546359497.A.6BF.html
https://www.ptt.cc/bbs/Beauty/M.1546368598.A.746.html
https://www.ptt.cc/bbs/Beauty/M.1546381048.A.A2C.html
https://www.ptt.cc/bbs/Beauty/M.1546392749.A.AFF.html
https://www.ptt.cc/bbs/Beauty/M.1546394016.A.5D6.html
https://www.ptt.cc/bbs/Beauty/M.1546395058.A.B73.html
https://www.ptt.cc/bbs/Beauty/M.1546396836.A.F11.html
https://www.ptt.cc/bbs/Be

IndexError: list index out of range

In [24]:
Keyword('媽',801,815)

ok
https://www.ptt.cc/bbs/Beauty/M.1564599079.A.245.html
https://www.ptt.cc/bbs/Beauty/M.1564616571.A.A83.html
https://www.ptt.cc/bbs/Beauty/M.1564619364.A.16B.html
https://www.ptt.cc/bbs/Beauty/M.1564625144.A.BC6.html
https://www.ptt.cc/bbs/Beauty/M.1564625624.A.632.html
https://www.ptt.cc/bbs/Beauty/M.1564638981.A.8F0.html
https://www.ptt.cc/bbs/Beauty/M.1564639882.A.A4C.html
https://www.ptt.cc/bbs/Beauty/M.1564642395.A.355.html
https://www.ptt.cc/bbs/Beauty/M.1564652719.A.49F.html
https://www.ptt.cc/bbs/Beauty/M.1564653243.A.126.html
https://www.ptt.cc/bbs/Beauty/M.1564658913.A.2FB.html
https://www.ptt.cc/bbs/Beauty/M.1564659819.A.CE8.html
https://www.ptt.cc/bbs/Beauty/M.1564660713.A.1AA.html
https://www.ptt.cc/bbs/Beauty/M.1564661726.A.C92.html
https://www.ptt.cc/bbs/Beauty/M.1564662447.A.8FD.html
https://www.ptt.cc/bbs/Beauty/M.1564664442.A.D49.html
https://www.ptt.cc/bbs/Beauty/M.1564668630.A.6A6.html
https://www.ptt.cc/bbs/Beauty/M.1564669228.A.6A8.html
https://www.ptt.cc/bbs/Be

MissingSchema: Invalid URL '板橋國中': No schema supplied. Perhaps you meant http://板橋國中?

In [25]:
Keyword('maroka',1231,1231)

https://www.ptt.cc/bbs/Beauty/M.1577744345.A.DD3.html
https://www.ptt.cc/bbs/Beauty/M.1577755372.A.3A3.html
https://www.ptt.cc/bbs/Beauty/M.1577762157.A.66E.html
https://www.ptt.cc/bbs/Beauty/M.1577762269.A.5AE.html
https://www.ptt.cc/bbs/Beauty/M.1577763932.A.6BE.html
https://www.ptt.cc/bbs/Beauty/M.1577766013.A.217.html
https://www.ptt.cc/bbs/Beauty/M.1577769714.A.ADA.html
https://www.ptt.cc/bbs/Beauty/M.1577770506.A.AFB.html
https://www.ptt.cc/bbs/Beauty/M.1577781747.A.233.html
https://www.ptt.cc/bbs/Beauty/M.1577784650.A.818.html
https://www.ptt.cc/bbs/Beauty/M.1577784823.A.AA3.html
https://www.ptt.cc/bbs/Beauty/M.1577785222.A.421.html
https://www.ptt.cc/bbs/Beauty/M.1577786704.A.C29.html
https://www.ptt.cc/bbs/Beauty/M.1577786904.A.72F.html
https://www.ptt.cc/bbs/Beauty/M.1577788966.A.218.html
https://www.ptt.cc/bbs/Beauty/M.1577801740.A.E41.html
